### Multi-turn 1. ConversationBufferWindowMemory

In [3]:
import os
import torch
import dotenv
from operator import itemgetter
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

dotenv.load_dotenv()

app = App(token=os.environ.get("SLACK_BOT_TOKEN"))

llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

In [4]:
template = """ You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: {history}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)

# Function to create a new LLMChain with a fresh memory
def create_chatgpt_chain():
    memory = ConversationBufferWindowMemory(k=3)
    return LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )

# Create a new LLMChain instance with fresh memory
chatgpt_chain = create_chatgpt_chain()

In [5]:
#Event handler for Slack
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    message = body["event"]['text']

    output = chatgpt_chain.predict(human_input = message)  
    say(output) 

#Message handler for Slack
@app.message(".*")
def message_handler(message, say, logger):
    print(message)
    
    output = chatgpt_chain.predict(human_input = message['text'])   
    say(output)

# Start your app
if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()

Bolt app is running!
{'user': 'U07D3QJHNKS', 'type': 'message', 'ts': '1721722255.682119', 'client_msg_id': 'af7f12bb-a592-4b3f-bb2b-3b68b0443469', 'text': 'ㅋㅋㅋ', 'team': 'T07CTLV9XPB', 'blocks': [{'type': 'rich_text', 'block_id': 'Q1oxi', 'elements': [{'type': 'rich_text_section', 'elements': [{'type': 'text', 'text': 'ㅋㅋㅋ'}]}]}], 'channel': 'D07C8FB6554', 'event_ts': '1721722255.682119', 'channel_type': 'im'}


> Entering new LLMChain chain...
Prompt after formatting:
 You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: 

#Human: ㅋㅋㅋ

#Assistant: 



> Finished chain.
{'user': 'U07D3QJHNKS', 'type': 'mess

### Multi-turn 2. RunnableWithMessageHistory

In [1]:
import os
import torch
import dotenv
from operator import itemgetter
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.base import BaseCallbackHandler
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory

dotenv.load_dotenv()

app = App(token=os.environ.get("SLACK_BOT_TOKEN"))

llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

c:\Users\shiny\AppData\Local\anaconda3\envs\llm_pj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
template = """ You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: {history}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(
    input_variables=["history", "human_input"], 
    template=template
)

chatgpt_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        output_parser=StrOutputParser(),
        verbose=True
    )

c:\Users\shiny\AppData\Local\anaconda3\envs\llm_pj\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [3]:
store = {}                                              # 세션 기록을 저장할 딕셔너리

def get_session_history(session_ids):
    print(f"[Conversation Session ID]: {session_ids}")
    if session_ids not in store:                        # 세션 ID가 store에 없는 경우                               
        store[session_ids] = ChatMessageHistory()       # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
    return store[session_ids]                           # 해당 세션 ID에 대한 세션 기록 반환


chain_with_history = RunnableWithMessageHistory(       
    chatgpt_chain,
    get_session_history,                               # 세션 기록을 가져오는 함수
    input_messages_key="human_input",                  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history"                     # 기록 메시지의 키
)

In [ ]:
# Slack 이벤트 핸들러
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    session_id = body['event']['channel']
    message = body['event']['text']
    history = get_session_history(session_id)
    
    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": message, "history": history},
        config={"configurable": {"session_id": session_id}}
    )

    # 응답을 문자열로 변환하여 슬랙에 전송
    say(response)

# Slack 메시지 핸들러
@app.message(".*")
def message_handler(message, say, logger):
    session_id = message['channel']
    text = message['text']
    history = get_session_history(session_id)

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": text, "history": history},
        config={"configurable": {"session_id": session_id}}
    )

    # 응답을 문자열로 변환하여 슬랙에 전송
    say(response)

if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()

### ⚒️ Solving the problem

In [4]:
store = {}  # 세션 기록을 저장할 딕셔너리

def get_session_history(session_id):                                # 주어진 세션 ID에 대한 대화 기록을 가져옵니다. 세션 ID가 처음 요청된 경우에는 새로운 대화 기록 객체를 생성하여 저장합니다.
    print(f"[Conversation Session ID]: {session_id}")
    if session_id not in store:                                     # 세션 ID가 store에 없는 경우                               
        store[session_id] = ChatMessageHistory()                    # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
    return store[session_id]                                        # 해당 세션 ID에 대한 세션 기록 반환

def format_history(history):                                        # 대화 기록 객체를 포맷하여 문자열로 변환합니다.
    messages = []
    for msg in history.messages:
        if isinstance(msg, HumanMessage):                           # 메시지가 HumanMessage 타입인 경우
            messages.append(f"Human: {msg.content}")                # 'Human: ' 접두어와 함께 메시지 내용 추가
        elif isinstance(msg, AIMessage):                            # 메시지가 AIMessage 타입인 경우
            messages.append(f"AI: {msg.content}")                   # 'AI: ' 접두어와 함께 메시지 내용 추가
    return "\n".join(messages)                                      # 메시지 리스트를 줄 바꿈으로 결합하여 문자열 반환

chain_with_history = RunnableWithMessageHistory(
    chatgpt_chain,
    get_session_history,                                            # 세션 기록을 가져오는 함수
    input_messages_key="human_input",                               # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history"                                  # 기록 메시지의 키
)

In [5]:
# Slack 이벤트 핸들러
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    session_id = body['event']['channel']                           # 이벤트가 발생한 채널 ID를 세션 ID로 사용
    message = body['event']['text']                                 # 메시지 내용을 가져옵니다.
    history = get_session_history(session_id)                       # 해당 세션 ID의 대화 기록을 가져옵니다.
    formatted_history = format_history(history)                     # 대화 기록을 포맷하여 문자열로 변환

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": message, "history": formatted_history},
        config={"configurable": {"session_id": session_id}}
    )

    # 응답 딕셔너리에서 'text' 필드만 추출하여 슬랙에 전송합니다.
    response_text = response.get('text', '응답을 생성하는 데 문제가 발생했습니다.')
    say(response_text)

# Slack 메시지 핸들러
@app.message(".*")
def message_handler(message, say, logger):
    session_id = message['channel']
    text = message['text']
    history = get_session_history(session_id)
    formatted_history = format_history(history)

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": text, "history": formatted_history},
        config={"configurable": {"session_id": session_id}}
    )

    # 응답을 문자열로 변환하여 슬랙에 전송
    response_text = response.get('text', '응답을 생성하는 데 문제가 발생했습니다.')
    say(response_text)

if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()

Bolt app is running!
[Conversation Session ID]: D07C8FB6554
[Conversation Session ID]: D07C8FB6554


> Entering new LLMChain chain...
Prompt after formatting:
 You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: []

#Human: 안녕! 나는 수현이라고해!

#Assistant: 



> Finished chain.
[Conversation Session ID]: D07C8FB6554
[Conversation Session ID]: D07C8FB6554


> Entering new LLMChain chain...
Prompt after formatting:
 You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also

### ✨ Summarization Chain

* Multi-turn 1. ConversationBufferWindowMemory으로 구현